In [1]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import gaussian_kde

In [2]:
df = pd.read_csv("datasets/cleaned_aqi_data_local.csv")
print(df.columns)
print(df.info())

Index(['date', 'city', 'pm10', 'pm2_5', 'carbon_monoxide', 'nitrogen_dioxide',
       'sulphur_dioxide', 'ozone', 'uv_index_clear_sky', 'uv_index', 'dust',
       'aerosol_optical_depth', 'us_aqi'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54792 entries, 0 to 54791
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   54792 non-null  object 
 1   city                   54792 non-null  object 
 2   pm10                   54792 non-null  float64
 3   pm2_5                  54792 non-null  float64
 4   carbon_monoxide        54792 non-null  float64
 5   nitrogen_dioxide       54792 non-null  float64
 6   sulphur_dioxide        54792 non-null  float64
 7   ozone                  54792 non-null  float64
 8   uv_index_clear_sky     54792 non-null  float64
 9   uv_index               54792 non-null  float64
 10  dust                   54792 non-null  fl

In [3]:
desc = df.describe().T[["count","min","max","mean","50%"]].rename(columns={"50%":"median"})
desc["mode"] = df.mode(numeric_only=True).iloc[0]
print(desc)
import dataframe_image as dfi
dfi.export(desc, "desc_table.png", table_conversion="matplotlib")

                         count         min         max        mean     median  \
pm10                   54792.0    0.800000   288.70000   52.192072   42.30000   
pm2_5                  54792.0    0.500000   259.80000   38.466035   29.80000   
carbon_monoxide        54792.0  101.000000  4134.00000  410.563659  324.00000   
nitrogen_dioxide       54792.0    0.000000   147.60000   16.175161   11.80000   
sulphur_dioxide        54792.0    0.000000    46.10000    7.997060    7.10000   
ozone                  54792.0    0.000000   281.00000   75.073405   64.00000   
uv_index_clear_sky     54792.0    0.000000    13.95000    1.885577    0.00000   
uv_index               54792.0    0.000000    11.85000    1.383398    0.00000   
dust                   54792.0    0.000000   290.00000    6.077201    2.00000   
aerosol_optical_depth  54792.0    0.030000     3.33000    0.560338    0.49000   
us_aqi                 54792.0   13.506945   241.66666   99.823157   93.21365   

                           

In [4]:
numeric_columns = df.select_dtypes("number").columns

### Univariate Distribution Analysis (Histograms for numeric columns)

In [ ]:
output_dir = "./screenshots/histograms_kde"
os.makedirs(output_dir, exist_ok=True)

for col in numeric_columns:
    data = df[col]
    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=data,
        name=col,
        histnorm="probability density",
        opacity=0.75,
    ))
    kde = gaussian_kde(data)
    x_grid = np.linspace(data.min(), data.max(), 500)
    fig.add_trace(go.Scatter(
        x=x_grid,
        y=kde(x_grid),
        mode="lines",
        name=f"KDE",
        line=dict(width=2),
    ))
    fig.update_layout(
        title_text=f"Distribution of {col}",
        xaxis_title=col,
        yaxis_title="Density",
        bargap=0.1,
        template="plotly_white",
    )

    fname = os.path.join(output_dir, f"{col}_hist_kde.png")
    fig.write_image(fname, format="png", engine="kaleido")

    fig.show()

### Spearman-correlation heatmap

In [ ]:
corr = df[numeric_columns].corr(method="spearman")

pastel_scale = [
    [0.0, "rgb(198,232,212)"],   # light green
    [0.5, "rgb(255,255,204)"],   # pale yellow
    [1.0, "rgb(245,105,108)"]    # stronger coral‑red
]

fig = go.Figure(data=go.Heatmap(
    z=corr.values,
    x=corr.columns,
    y=corr.index,
    zmin=-1, zmax=1, zmid=0,
    colorscale=pastel_scale,
    text=corr.round(2).values,
    texttemplate="%{text}",
    textfont=dict(size=12),
    colorbar=dict(
        title="ρ",
        lenmode="fraction", len=0.8,
    )
))

fig.update_layout(
    title="Spearman Correlation Matrix",
    width=1000,
    height=850,
    template="plotly_white",
    xaxis_nticks=len(corr.columns),
    yaxis_nticks=len(corr.index),
    margin=dict(t=100, l=200, b=200)
)

fname = os.path.join(output_dir, f"spearman_corr.png")
fig.show()

### Pairwise scatter grid

In [5]:
from plotly.colors import qualitative

In [6]:
df_sample = df.sample(5000, random_state=42).reset_index(drop=True)
cities = df_sample["city"].astype("category")
codes = cities.cat.codes
palette = qualitative.Safe[:3]
colors  = [palette[i % len(palette)] for i in codes]
dimensions = []
for col in numeric_columns:
    dim_dict = {"label": col, "values": df_sample[col]}
    dimensions.append(dim_dict)

In [9]:
# for i, city in enumerate(df_sample["city"].unique()):
# sub = df_sample[df_sample["city"] == city]
fig = go.Figure(go.Splom(
    dimensions=dimensions,
    marker=dict(color=colors[2], size=4)
))

fig.update_layout(
    title=f"Pairwise Relationship",
    width=1200,
    height=1200,
    margin=dict(l=100, r=100, t=100, b=100),
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",
    font=dict(size=8)
)

fig.show()

In [ ]:
for i, city in enumerate(df_sample["city"].unique()):
    sub = df_sample[df_sample["city"] == city]
    fig = go.Figure(go.Splom(
        dimensions=dimensions,
        marker=dict(color=colors[i], size=4)
    ))

    fig.update_layout(
        title=f"Pairwise Relationship by {city.title()}",
        width=1200,
        height=1200,
        margin=dict(l=100, r=100, t=100, b=100),
        template="plotly_white",
        plot_bgcolor="rgba(0, 0, 0, 0)",
        font=dict(size=8)
    )

    fig.show()

### Time series decomposition

In [ ]:
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import STL

In [ ]:
df_copy = df.__deepcopy__()

In [ ]:
df_copy["date"] = pd.to_datetime(df_copy["date"])
df_copy.set_index("date", inplace=True)

In [ ]:
col = "pm2_5"

In [ ]:
for city in df_copy["city"].unique():
    df_city = df_copy[df_copy["city"]==city]
    series = df_city[col]
    stl = STL(series, period=24, robust=True)
    res = stl.fit()

    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, subplot_titles=["Trend", "Seasonal", "Residual"])

    fig.add_trace(go.Scatter(
        x=res.trend.index, y=res.trend, mode="lines", name="Trend",
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=res.seasonal.index, y=res.seasonal, mode="lines", name="Seasonal",
    ), row=2, col=1)
    fig.add_trace(go.Scatter(
        x=res.resid.index, y=res.resid, mode="lines", name="Residual",
    ), row=3, col=1)

    fig.update_layout(
        height=1000,
        width=800,
        title=f"STL Decomposition of PM2.5 for {city.title()}",
        template="plotly_white"
    )

    fig.show()

### PM2.5 autocorrelation check

In [ ]:
from statsmodels.tsa.stattools import acf

In [ ]:
max_lag = 30

for city in df_copy["city"].unique():
    df_city = df_copy[df_copy["city"]==city]
    acf_vals = acf(df_city[col], nlags=max_lag, fft=True)
    
    fig = go.Figure(
        data=go.Bar(
            x=np.arange(len(acf_vals)),
            y=acf_vals,
            marker_color='steelblue'
        )
    )

    fig.add_hline(y=0, line_dash="dash", line_color="black")
    fig.update_layout(
        title=f"Autocorrelation of Daily PM2.5 for {city.title()}",
        xaxis_title="Lag (days)",
        yaxis_title="ACF",
        template="plotly_white",
        width=700,
        height=400
    )

    fig.show()

### City-level aggregation

In [ ]:
important_cols = ["pm2_5", "pm10", "us_aqi"]

In [ ]:
city_means = df.groupby("city")[important_cols].mean().sort_values(by="pm2_5")

In [ ]:
fig = make_subplots(
    rows=1,
    cols=3,
    shared_yaxes=False,
    subplot_titles=["Average PM2.5", "Average PM10", "Average AQI (US)"]
)

for i, col in enumerate(important_cols):
    fig.add_trace(go.Bar(
        x=(city_means.index).str.title(),
        y=city_means[col],
        name=col.upper().replace("_", " "),
        text=city_means[col].round(1),
        textposition="outside",
        textfont=dict(size=8)
    ), row=1, col=i+1)

fig.update_layout(
    title="Average Pollutant Levels by City (PM2.5, PM10)",
    template="plotly_white",
    width=1000,
    height=400,
    showlegend=False,
    margin=dict(t=80, l=40, r=40, b=40)
)

fig.update_xaxes(tickangle=-45)

fig.show()

### Cluster and silhouette analysis

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [ ]:
X = StandardScaler().fit_transform(df[[col for col in numeric_columns if col != "us_aqi"]].values)
y = df["us_aqi"].values

In [ ]:
scores = []
for k in range(2, 11):
    labels = KMeans(n_clusters=k, random_state=42).fit_predict(X)
    scores.append(silhouette_score(X, labels))

In [ ]:
ks = range(2, 11)

In [ ]:
fig_curve = go.Figure(go.Scatter(
    x=list(ks),
    y=scores,
    mode="lines+markers",
    marker=dict(size=8),
    line=dict(width=2)
))
fig_curve.update_layout(
    title="Silhouette Analysis",
    xaxis_title="k",
    yaxis_title="Mean Silhouette Score",
    template="plotly_white",
    width=600,
    height=400
)
fig_curve.show()

In [ ]:
k_opt = ks[int(np.argmax(scores))]
kmeans = KMeans(n_clusters=k_opt, random_state=42).fit(X)
cluster_labels = kmeans.labels_
print(f"Optimal k = {k_opt}, mean silhouette = {max(scores):.3f}")

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca   = PCA(n_components=2, random_state=42)
X_2d  = pca.fit_transform(X)

In [ ]:
fig_2d = go.Figure(go.Scatter(
    x=X_2d[:, 0],
    y=X_2d[:, 1],
    mode='markers',
    marker=dict(
        color=cluster_labels,
        colorscale='Viridis',
        size=6,
        line=dict(width=0.4, color='black')
    ),
    text=[f"AQI: {v:.1f}" for v in y],
    hovertemplate="PC1=%{x:.2f}<br>PC2=%{y:.2f}<br>%{text}<extra>Cluster %{marker.color}</extra>"
))

fig_2d.update_layout(
    title=f"K‑means Clusters in PCA Space (k={k_opt})",
    xaxis_title="Principal Component 1",
    yaxis_title="Principal Component 2",
    template="plotly_white",
    width=700,
    height=600
)

fig_2d.show()